In [12]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# **Load the training data**

We load the training data.

In [13]:
import datetime
from implementations import *

data_load = load_data("x_train.csv")
pred = load_data("y_train.csv")
print(f"The data has {data_load.shape[0]} samples and {data_load.shape[1]} features !")

The data has 328135 samples and 321 features !


The first column is states and I create a new column to classify with the ranking of PIB between them, and replace each state number by their position in the ranking.

In [14]:
def replace(arr, old_values, new_values):
    result = arr.copy()

    for old_val, new_val in zip(old_values, new_values):
        result[result == old_val] = new_val

    return result

In [17]:
# Where put district of columbia ? super high PIB

new_rank_state = [27,46,20,34,1,16,23,42,0,4,9,38,40,5,19,30,
                 33,28,24,43,15,11,14,17,12,22,48,35,32,39,8,37,3,
                 12,45,7,29,25,6,44,26,47,18,2,31,50,13,10,41,21,49,51,52]
old_states_name =  [1,2,4,5,6,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,
                    29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,53,54,55,56,66,72]

ranking_column = replace(data_load[:,0], old_states_name, new_rank_state)

Next 5 columns are useless, they are about interview date and index of patient
## COLUMNS TO DELETE 2 3 4 5 7 8 9 10 12 14 19 20 21 22 23

## POSSIBLE MEAN 13 24 25 26 27 28 29 33 37 59

The 6th column is DISPCODE, so if interview was complete or not, I change value to 1 if complete 0 if not.

In [19]:
data_load[:,6] = replace(data_load[:,6], [1100,1200], [1,0])
data_load[:,13] = replace(data_load[:,13], [0,1], [1,2])
data_load[:,24] = replace(data_load[:,24], [1,2,7,9], [0,1,np.nan,np.nan])
data_load[:,25] = replace(data_load[:,25], [77,99], [np.nan,np.nan])
data_load[:,26] = replace(data_load[:,26], [2,3,4,5,7,9], [0.75,0.5,0.25,0,np.nan,np.nan])

array_1 = [27,28,29]

for i in array_1 : 
    data_load[:,i] = replace(data_load[:,i], [88,77,99], [np.nan,np.nan,np.nan])

data_load[:,31] = replace(data_load[:,31], [3,7,9], [0,np.nan,np.nan])

array_2 = [30,32,34,35,36,38,39,40,41,42,43,44,45,46,47,48,53,54,55,56,57,61,64,65,66,67,68,69,70,71,72,73,74,76,87,95,96,100]
           
for i in array_2:
    data_load[:,i] = replace(data_load[:,i], [7,9], [np.nan,np.nan])
    

data_load[:,33] = replace(data_load[:,33], [1,2,3,4,7,8,9], [6,18,42,60,np.nan,120,np.nan])


data_load[:,37] = replace(data_load[:,37], [1,2,3,4,7,9], [6,18,42,60,np.nan,np.nan])
data_load[:,49] = replace(data_load[:,49], [98,99], [np.nan,np.nan])

array_3 = [51,52,58]

for i in array_3 : 
    data_load[:,i] = replace(data_load[:,i], [9], [np.nan])
    

9th to 13th column are bound together, they help getting the answer on the last column CELLFON3, so we can remove the 9th, 10th, 11th and 12th columns.
I create a new column to know if the survey is done on a cellphone or fixed phone.


Column 15,16,17 is super useful and tells us about the number of adults in the household, but they are not logic. Number of household where 1 adult < Number of household where 1 adult men

19th column to 24th bound together so only last one useful (24th)
We denote the value in the 24th column 0 if the person is a woman and doesnt have a fixed number to receive calls, 1 if she has one.
2 if the person is a man and doesnt have a fixed number to receive calls, 3 if he has one.


Column 25 is super interesting tells us the number of adults in the household and is similar to column 15 but seems more logic !
But the values are extremes we have to change them !


Column 30

In [39]:
data_load[:,49] = replace(data_load[:,49], old_edzdzzd, ZEEZEZ)

erer = [np.nan]
old_rerere = [9]

data_load[:,51] = replace(data_load[:,51], old_rerere, erer)
data_load[:,52] = replace(data_load[:,52], old_rerere, erer)

In [41]:
YUYU = [np.nan,np.nan]
old_ryuu = [88,99]

data_load[:,59] = replace(data_load[:,59], old_ryuu, YUYU)

oioi = [5,12.5,17.5,22.5,30,42.5,62.5,75,np.nan,np.nan]

old_oioi = [1,2,3,4,5,6,7,8,77,99] 

data_load[:,60] = replace(data_load[:,60], old_rerere, erer)

In [42]:
w = [np.nan,np.nan]
old_w = [7777,9999]

data_load[:,62] = replace(data_load[:,62], old_w, w)

In [43]:
print(data_load[:, 62])

[110. 200.  nan ... 320. 250. 150.]


In [51]:
indices = np.where(np.logical_and(data_load[:, 62] > 9000, data_load[:, 62] < 9999))
result = indices[0]
print(result)
print(data_load[1469, 62])
print(data_load[15558, 62])

[  1469   1747   1767   2151   2269   2284   4233   5266   6405   6775
   6783   7134   8372   9317   9532   9821   9900  10154  10897  11447
  11454  11825  12121  12588  12827  12889  12981  13009  13251  13894
  14195  14208  14439  14842  15187  15558  15869  15892  15901  16507
  16695  17692  18044  18079  18423  18994  19887  19928  20842  22777
  23525  24595  24811  24949  25139  26481  27377  27427  28719  29275
  30026  30303  30974  31039  31560  32096  32514  32552  33390  33510
  33572  35813  35911  36459  36655  36992  37294  37322  39068  39407
  39652  39906  39973  39992  40349  40678  41688  42071  42524  42662
  43192  43247  43451  43928  44796  45062  45351  46130  46233  46395
  47438  47544  48852  48954  49235  49970  50038  50271  50481  51322
  51403  51664  52190  52392  52872  53676  54134  54639  54644  54651
  54711  54786  55475  55881  56270  56517  57805  58186  58529  58774
  58900  59661  59979  59985  60737  61166  61347  61636  61942  61987
  6208

In [ ]:
indices = np.where(np.logical_and(data_load[:, 63] > 9000, data_load[:, 63] < 9999))
result = indices[0]
print(result)
print(data_load[1469, 62])
print(data_load[15558, 62])

In [66]:
#62 DEMANDER AVIS
#D'abord le replace et après :

def convert_value(x):
    if x >= 9000 :
        return int((x - 9000) * 2.20462)
    else:
        return x 

data_load[:, 62] = list(map(convert_value,(data_load[:, 62])))

In [80]:
indices = np.where(np.logical_and(data_load[:, 63] > 9000, data_load[:, 63] < 9999))
result = indices[0][0:10]
result
print(data_load[171, 63])
print(data_load[1767, 63])

59.055150000000005
65.748067


In [79]:
#63 LA MEME
def func(x):
    str_x = str(x)
    
    if x < 9000:          
        return np.floor(x/100)*12 + (x % 100)
    else: 
        return (x - 9000) * 0.393701

data_load[:, 63] = list(map(func,(data_load[:, 63])))

In [ ]:
print(data_load[171, 63])
print(data_load[1767, 63])

In [ ]:
###
[15,60,135,270,1080,2070,3600,np.nan,np.nan,np.nan]
[1,2,3,4,5,6,7,8,77,99]

In [ ]:
### A MONTRER AUSSI
def func3WEEKMONTH(x):
    x_str = str(x)
    if x_str[0] === "1":       
        return 4*int(x_str[-2:])
    else:
        return int(x_str[-2:])

data_load[:, 77] = np.vectorize(func3WEEKMONTH)(data_load[:, 77])

In [ ]:
### A MONTRER AUSSI
def func3334(x):
    x_str = str(x)
    if x_str[0] === "1":       
        return 30 *int(x_str[-2:])
    elif x_str[0] === "2":
        return (4.25*int(x_str[-2:])).astype(int)
    else 
        return int(x_str[-2:])

data_load[:, 81] = np.vectorize(func3334)(data_load[:, 81])
data_load[:, 82] = np.vectorize(func3334)(data_load[:, 82])
data_load[:, 83] = np.vectorize(func3334)(data_load[:, 83])
data_load[:, 84] = np.vectorize(func3334)(data_load[:, 84])
data_load[:, 85] = np.vectorize(func3334)(data_load[:, 85])
data_load[:, 86] = np.vectorize(func3334)(data_load[:, 86])

In [ ]:
####
data_load[:, 89] = np.vectorize(func3WEEKMONTH)(data_load[:, 89])

In [ ]:
#### 
def funcHEUREMIN(x):
    x_str = str(x)
    return int(x_str[0])*60 + int(x_str[-2:])
    

data_load[:, 90] = np.vectorize(funcHEUREMIN)(data_load[:, 90])

In [ ]:
####"

data_load[:, 92] = np.vectorize(func3WEEKMONTH)(data_load[:, 92])

In [ ]:
data_load[:, 93] = np.vectorize(funcHEUREMIN)(data_load[:, 93])

In [ ]:
data_load[:, 94] = np.vectorize(func3WEEKMONTH)(data_load[:, 94])

In [ ]:
####  97 
zzzzze = [0.5,0,np.nan,np.nan]
old_ezezez = [2,3,7,9]

In [ ]:
###99 
[2,3,4,5,7,8,9] => [0.75,0.5,0.25,0,np.nan,np.nan,np.nan]

We clean the data by removing the nan values by the mean of the rest of the feature

In [ ]:
data = np.ones(data_load.shape)
stds = np.array([])
for i in range(data.shape[1]):
    d, std = standardize_clean(data_load[:, i])
    data[:, i] = d
    stds = np.append(stds, std)
print(stds.shape)

We further clean the data by removing the features where the variance is zero since they are constants for all samples

In [ ]:
indices = np.where(stds != 0)
data_var = data[:, indices]
data_var = np.squeeze(data_var, axis = 1)
print(data_var.shape)

We also remove the 8 first features as the appear weird in the task of predicting a heart attack

In [ ]:
data_cleaned = data_var[:, 9:]
print(data_cleaned.shape)

We then separe the data to train on 60% of the total and to test it on the remaining 40% 

In [ ]:
train_size = np.floor(data_cleaned.shape[0] * 0.6).astype(int)
data_cross = data_cleaned[:train_size, :]
pred_cross = pred[:train_size]
data_test = data_cleaned[train_size:, :]
pred_test = pred[train_size:]
print(f"Cross shape : {data_cross.shape} ; Test shape : {data_test.shape}")

Now we build our models for linear regression

In [ ]:
y, tx = build_model_data(data_cross, pred_cross)
y_test, tx_test = build_model_data(data_test, pred_test)
print(f"The data has now {tx.shape[1]} features !")